# Imports

In [4]:
import polars as pl
from datetime import datetime
from pathlib import Path

# Config

In [5]:
path_data_raw = Path() / '..' / 'data'

# Polars Stuff

In [6]:
df = pl.DataFrame(
    {
        "integer": [1, 2, 3],
        "date": [
            datetime(2025, 1, 1),
            datetime(2025, 1, 2),
            datetime(2025, 1, 3),
        ],
        "float": [4.0, 5.0, 6.0],
        "string": ["a", "b", "c"],
    }
)

display(df)

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""
3,2025-01-03 00:00:00,6.0,"""c"""


In [8]:
df.write_csv(path_data_raw / "output.csv")
df_csv = pl.read_csv(path_data_raw / "output.csv")

In [10]:
df.select(pl.col('*'))

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""
3,2025-01-03 00:00:00,6.0,"""c"""


In [21]:
df.filter(
    # pl.col("date").is_between('2025-01-01', '2025-01-02'),
    pl.col("date").is_between(datetime(2025, 1, 1), datetime(2025, 1, 2))
)

integer,date,float,string
i64,datetime[μs],f64,str
1,2025-01-01 00:00:00,4.0,"""a"""
2,2025-01-02 00:00:00,5.0,"""b"""


## Group By

In [22]:
df2 = pl.DataFrame(
    {
        "x": range(8),
        "y": ["A", "A", "A", "B", "B", "C", "X", "X"],
    }
)

In [23]:
display(df2)

x,y
i64,str
0,"""A"""
1,"""A"""
2,"""A"""
3,"""B"""
4,"""B"""
5,"""C"""
6,"""X"""
7,"""X"""


In [30]:
df2.group_by("y", maintain_order=True).len()

y,len
str,u32
"""A""",3
"""B""",2
"""C""",1
"""X""",2


In [31]:
df2.group_by("y", maintain_order=True).agg(
    pl.col("*").count().alias("count"),
    pl.col("*").sum().alias("sum"),
)

y,count,sum
str,u32,i64
"""A""",3,3
"""B""",2,7
"""C""",1,5
"""X""",2,13
